In [ ]:
# Import all required libraries
# Read csv the dataframe
# Inspect the dataframe and clean if required
# Make new dataframe with selected feature columns
# Scale the features
# Test train split (Randon seed, shuffle, stratify)


# Create a baseline MLP model and fit the training data
# Evaluate the model results of the baseline
# Run Controlled Experiments to improve model performance
# run final MLP model using the optimal parameters and regularization techniques found from our controlled experiment
# Predict the test data
# Add visualisations


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [30]:
ttc_initial=pd.read_csv("C:/Users/aakav/dsi/ml13-ttc-delays-project/src_files/ttc_delays_final_df.csv")

In [31]:
ttc_initial.head()

,date,year,month,day_of_month,weekday_num,time,hour,minute,day,day_of_week,...,delay_category,min_delay,location_bound,line,major_delay_flag,rush_hour_flag,controllable_delay_flag,season,major_event_flag,major_event_desc
0,2014-01-01,2014,1,1,3,00:21,0,21,Wednesday,3,...,Passenger,55,W,BD,1,0,0,Winter,0,NaN
1,2014-01-01,2014,1,1,3,02:06,2,6,Wednesday,3,...,Passenger,3,W,BD,0,0,0,Winter,0,NaN
2,2014-01-01,2014,1,1,3,03:10,3,10,Wednesday,3,...,Passenger,3,W,BD,0,0,0,Winter,0,NaN
3,2014-01-01,2014,1,1,3,03:20,3,20,Wednesday,3,...,Passenger,5,S,YU,0,0,1,Winter,0,NaN
4,2014-01-01,2014,1,1,3,08:48,8,48,Wednesday,3,...,Staff,5,E,BD,0,1,1,Winter,0,NaN


In [32]:
# Select features and target variable and create new dataframe
# month	day_of_month	hour	day_of_week	delay_category	line	major_delay_flag

ttc=ttc_initial[['month','day_of_month','weekday_num','hour','day_of_week','delay_category','line','rush_hour_flag','season','major_delay_flag']]

In [33]:
ttc.head()

,month,day_of_month,weekday_num,hour,day_of_week,delay_category,line,rush_hour_flag,season,major_delay_flag
0,1,1,3,0,3,Passenger,BD,0,Winter,1
1,1,1,3,2,3,Passenger,BD,0,Winter,0
2,1,1,3,3,3,Passenger,BD,0,Winter,0
3,1,1,3,3,3,Passenger,YU,0,Winter,0
4,1,1,3,8,3,Staff,BD,1,Winter,0


In [34]:
# One-hot encode categorical variables
from sklearn.preprocessing import OneHotEncoder
categorical_features = ['delay_category', 'line', 'season']
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_features = encoder.fit_transform(ttc[categorical_features])
encoded_feature_names = encoder.get_feature_names_out(categorical_features)

## CHANGE 1
#encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names) 
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names, index=ttc.index)

ttc_encoded = pd.concat([ttc.drop(columns=['delay_category', 'line', 'season']), encoded_df], axis=1)
ttc_encoded

,month,day_of_month,weekday_num,hour,day_of_week,rush_hour_flag,major_delay_flag,delay_category_Passenger,delay_category_Staff,delay_category_Technical,delay_category_Weather,line_SHP,line_YU,season_Spring,season_Summer,season_Winter
0,1,1,3,0,3,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,3,2,3,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,1,3,3,3,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,1,3,3,3,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1,1,3,8,3,1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75499,9,30,2,20,2,0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
75500,9,30,2,20,2,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75501,9,30,2,20,2,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75502,9,30,2,21,2,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# Feature Scaling

# Create a copy of the original 'ttc' dataframe to ensure we're not modifying the original data
standardized_ttc = ttc_encoded.copy()

# Specify the columns that we do NOT want to scale
columns_to_exclude = ['rush_hour_flag','delay_category_Passenger','delay_category_Staff',
                      'delay_category_Technical','delay_category_Weather','line_SHP','line_YU',
                      'season_Spring','season_Summer','season_Winter','major_delay_flag']

# Select the columns that we want to scale by excluding the 'id' and 'diagnosis' columns
# This will return a list of the numeric columns we need to scale
columns_to_scale = standardized_ttc.columns.difference(columns_to_exclude)

# Initialize the StandardScaler to standardize the selected numeric columns
scaler = StandardScaler()

## CHANGE 2
#standardized_ttc[columns_to_scale] = scaler.fit_transform(ttc[columns_to_scale])
standardized_ttc[columns_to_scale] = scaler.fit_transform(standardized_ttc[columns_to_scale])

# Output the standardized dataframe with the scaled numeric columns
standardized_ttc


,month,day_of_month,weekday_num,hour,day_of_week,rush_hour_flag,major_delay_flag,delay_category_Passenger,delay_category_Staff,delay_category_Technical,delay_category_Weather,line_SHP,line_YU,season_Spring,season_Summer,season_Winter
0,-1.519599,-1.683939,-0.031463,-2.142370,-0.031463,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-1.519599,-1.683939,-0.031463,-1.813028,-0.031463,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-1.519599,-1.683939,-0.031463,-1.648357,-0.031463,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-1.519599,-1.683939,-0.031463,-1.648357,-0.031463,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,-1.519599,-1.683939,-0.031463,-0.825001,-0.031463,1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75499,0.781520,1.607590,-0.562198,1.151054,-0.562198,0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
75500,0.781520,1.607590,-0.562198,1.151054,-0.562198,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75501,0.781520,1.607590,-0.562198,1.151054,-0.562198,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75502,0.781520,1.607590,-0.562198,1.315725,-0.562198,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
#set the seed
np.random.seed(1)

#split the data
standardized_ttc_train, standardized_ttc_test =train_test_split(
    standardized_ttc, 
    test_size=0.25, 
    random_state=42, 
    shuffle=True, 
    stratify=standardized_ttc['major_delay_flag'])

In [37]:
standardized_ttc_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56628 entries, 43655 to 52367
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   month                     56628 non-null  float64
 1   day_of_month              56628 non-null  float64
 2   weekday_num               56628 non-null  float64
 3   hour                      56628 non-null  float64
 4   day_of_week               56628 non-null  float64
 5   rush_hour_flag            56628 non-null  int64  
 6   major_delay_flag          56628 non-null  int64  
 7   delay_category_Passenger  56628 non-null  float64
 8   delay_category_Staff      56628 non-null  float64
 9   delay_category_Technical  56628 non-null  float64
 10  delay_category_Weather    56628 non-null  float64
 11  line_SHP                  56628 non-null  float64
 12  line_YU                   56628 non-null  float64
 13  season_Spring             56628 non-null  float64
 14  season_

In [38]:
standardized_ttc_train["major_delay_flag"].value_counts(normalize=True)

major_delay_flag
0    0.845765
1    0.154235
Name: proportion, dtype: float64

In [ ]:
standardized_ttc_train.head()

,month,day_of_month,weekday_num,hour,day_of_week,rush_hour_flag,major_delay_flag,delay_category_Passenger,delay_category_Staff,delay_category_Technical,delay_category_Weather,line_SHP,line_YU,season_Spring,season_Summer,season_Winter
43655,-1.519599,0.018576,-1.623669,-0.330987,-1.623669,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1188,-0.944319,-1.456937,-1.092934,-0.825001,-1.092934,1,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
22904,1.644439,-0.208426,1.030008,-0.825001,1.030008,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2862,-0.081400,0.245578,-0.031463,0.986383,-0.031463,0,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
53980,-0.656679,0.132077,-1.092934,-0.825001,-1.092934,1,0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


## Create a baseline MLP Model with dropout and dense layers

Our initial idea was to conduct a logistic regression with a dense layer, but upon further research, we came across the Multilayer Perceptron (MLP) Model. At a high level, a logistic regression only has 1 latyer, but an MLP can have several hidden layers. 


In [43]:
# Separate Features (X) and Target (Y)

X_train = standardized_ttc_train.drop(columns='major_delay_flag')
y_train = standardized_ttc_train['major_delay_flag']

X_test = standardized_ttc_test.drop(columns='major_delay_flag')
y_test = standardized_ttc_test['major_delay_flag']

In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=10,
          batch_size=32,
          #validation_split=0.2
          class_weight={0:1, 1:3})  # adjust for imbalance

c:\Users\aakav\miniconda3\envs\dsi_participant\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7807 - loss: 0.8350 - val_accuracy: 0.8067 - val_loss: 0.5014
Epoch 2/10
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7956 - loss: 0.8039 - val_accuracy: 0.8121 - val_loss: 0.4900
Epoch 3/10
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7978 - loss: 0.7991 - val_accuracy: 0.7955 - val_loss: 0.5130
Epoch 4/10
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8002 - loss: 0.7940 - val_accuracy: 0.7883 - val_loss: 0.4929
Epoch 5/10
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7840 - loss: 0.7992 - val_accuracy: 0.8119 - val_loss: 0.4919
Epoch 6/10
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7984 - loss: 0.7984 - val_accuracy: 0.8180 - val_loss: 0.4851
Epoch 7/10
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8028 - loss: 0.7984 - val_accuracy: 0.8012 - val_loss: 0.5002
Epoch 8/10
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7907 - loss: 0.7871 -

In [73]:
# Evaluate on test set
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


590/590 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8081 - loss: 0.4970
Test Loss: 0.4985
Test Accuracy: 0.8075


In [79]:
from sklearn.metrics import classification_report

y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))


590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
              precision    recall  f1-score   support

           0       0.86      0.92      0.89     15965
           1       0.28      0.16      0.21      2911

    accuracy                           0.81     18876
   macro avg       0.57      0.54      0.55     18876
weighted avg       0.77      0.81      0.79     18876



# Running Controlled Experiments to optimize model accuracy

Create a loop that tests the following permutations:
- Regularization technique -  Model with or without a dropout layer after each dense layer.
- Hyperparameter options - Modes with 64, 128 filters.

This will give us 8 model options to compare and choose as out final model to run a SHAP amalysis with.

In [76]:
# Define parameter grid
dense_sizes = [64, 128]
dropout_options = [True, False]

results = []

for dense_size in dense_sizes:
    for dropout1 in dropout_options:
        for dropout2 in dropout_options:
            
            # Build model_test
            model_test = Sequential()
            model_test.add(Dense(dense_size, activation='relu', input_shape=(X_train.shape[1],)))
            if dropout1:
                model_test.add(Dropout(0.3))
            
            model_test.add(Dense(64, activation='relu'))
            if dropout2:
                model_test.add(Dropout(0.3))
            
            model_test.add(Dense(1, activation='sigmoid')) # Binary classification
            
            # Compile
            model_test.compile(optimizer='adam',
                          loss='binary_crossentropy',
                          metrics=['accuracy'])
            
            # Train
            model_test.fit(X_train, y_train,
                      validation_data=(X_test, y_test),
                      epochs=5, #reduced epochs for speed
                      batch_size=32,
                      class_weight={0:1, 1:3} #adjust for imbalance
                      )
            
            # Evaluate
            test_loss, test_acc = model_test.evaluate(X_test, y_test, verbose=0)
            
            # Save results
            results.append({
                'dense_layer_1': dense_size,
                'dropout_1': dropout1,
                'dropout_2': dropout2,
                'test_loss': round(test_loss, 4),
                'test_accuracy (%)': round(test_acc * 100, 2)
            })


c:\Users\aakav\miniconda3\envs\dsi_participant\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.7828 - loss: 0.8286 - val_accuracy: 0.8088 - val_loss: 0.4903
Epoch 2/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.7929 - loss: 0.8022 - val_accuracy: 0.7869 - val_loss: 0.4845
Epoch 3/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.7931 - loss: 0.7972 - val_accuracy: 0.7612 - val_loss: 0.4962
Epoch 4/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.7805 - loss: 0.7991 - val_accuracy: 0.8017 - val_loss: 0.5049
Epoch 5/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.7898 - loss: 0.8000 - val_accuracy: 0.8074 - val_loss: 0.4982


c:\Users\aakav\miniconda3\envs\dsi_participant\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.7854 - loss: 0.8269 - val_accuracy: 0.7865 - val_loss: 0.5092
Epoch 2/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.7830 - loss: 0.8047 - val_accuracy: 0.7956 - val_loss: 0.4807
Epoch 3/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.7877 - loss: 0.7913 - val_accuracy: 0.8015 - val_loss: 0.4952
Epoch 4/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.7901 - loss: 0.7889 - val_accuracy: 0.7875 - val_loss: 0.4961
Epoch 5/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.7734 - loss: 0.7965 - val_accuracy: 0.7914 - val_loss: 0.5094


c:\Users\aakav\miniconda3\envs\dsi_participant\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Epoch 1/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.7849 - loss: 0.8165 - val_accuracy: 0.7655 - val_loss: 0.4943
Epoch 2/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7721 - loss: 0.7938 - val_accuracy: 0.7806 - val_loss: 0.4893
Epoch 3/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.7865 - loss: 0.7959 - val_accuracy: 0.7704 - val_loss: 0.5172
Epoch 4/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.7846 - loss: 0.7942 - val_accuracy: 0.7640 - val_loss: 0.5146
Epoch 5/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.7868 - loss: 0.7930 - val_accuracy: 0.7561 - val_loss: 0.4794
Epoch 1/5


c:\Users\aakav\miniconda3\envs\dsi_participant\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1770/1770 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7858 - loss: 0.8150 - val_accuracy: 0.8168 - val_loss: 0.4935
Epoch 2/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7927 - loss: 0.7918 - val_accuracy: 0.7571 - val_loss: 0.5163
Epoch 3/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7819 - loss: 0.7940 - val_accuracy: 0.7959 - val_loss: 0.5043
Epoch 4/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7859 - loss: 0.7977 - val_accuracy: 0.7433 - val_loss: 0.5126
Epoch 5/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7861 - loss: 0.7818 - val_accuracy: 0.7890 - val_loss: 0.4830
Epoch 1/5


c:\Users\aakav\miniconda3\envs\dsi_participant\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1770/1770 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7638 - loss: 0.8188 - val_accuracy: 0.8005 - val_loss: 0.4817
Epoch 2/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7809 - loss: 0.7998 - val_accuracy: 0.7470 - val_loss: 0.4995
Epoch 3/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7637 - loss: 0.7977 - val_accuracy: 0.7917 - val_loss: 0.4926
Epoch 4/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7811 - loss: 0.7976 - val_accuracy: 0.7768 - val_loss: 0.4913
Epoch 5/5
1770/1770 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7807 - loss: 0.7905 - val_accuracy: 0.7774 - val_loss: 0.5020


In [77]:
# Convert experiment results to DataFrame and compare
results_df = pd.DataFrame(results)
print(results_df)


   dense_layer_1  dropout_1  dropout_2  test_loss  test_accuracy (%)
0             64       True       True     0.4982              80.74
1             64       True      False     0.5094              79.14
2             64      False       True     0.4962              76.76
3             64      False      False     0.5000              76.26
4            128       True       True     0.4945              79.60
5            128       True      False     0.4794              75.61
6            128      False       True     0.4830              78.90
7            128      False      False     0.5020              77.74


Our baseline model (called model) turns out to be the best performing model.

## Use Shap interprettion to understand feature 

There are two types of SHAP methods: shap.kmeans & shap.sample - will run both and compare. 